In [ ]:
import sys, os
repo_root = os.path.abspath(os.path.join(".."))

if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

print("PYTHONPATH set to:", repo_root)

sys.path.append('../')
sys.path.append('../cmouse/')
sys.path.append('../mouse_cnn/')
from anatomy import *
from architecture import *
%matplotlib inline

from config import *
# from network import Network
import torch.nn as nn
import torch.optim as optim

import torch
from whisker.multimodal_model_pool import MultimodalMouseModelPool
# from network import save_network_to_pickle
from network import load_network_from_pickle
from whisker.preprocess_dataset import PreprocessedTrialDataset

from mousenet_complete_pool import MouseNetCompletePool
from whisker.clip_train import clip_loss
from torch.utils.data import DataLoader

import zipfile
import os
import shutil
from pathlib import Path
import pandas as pd
from PIL import Image
import numpy as np
from torch.utils.data import DataLoader
import json

PYTHONPATH set to: c:\Users\canda\Documents\mousenet\Mouse_CNN\mousenet


In [4]:
# architecture = Architecture()
# anet = gen_anatomy(architecture)

# network = Network()
# network.construct_from_anatomy(anet, architecture)
# save_network_to_pickle(network, 'network_test.pkl')

network = load_network_from_pickle('network_test.pkl')
visual_net = MouseNetCompletePool(network)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultimodalMouseModelPool(visual_net=visual_net).to(device)

preprocess_dataset = False

{'inputLGNd': (0, 64, 0, 64), 'LGNdVISp4': (0, 64, 0, 51), 'VISp4VISp2/3': (0, 78, 0, 50), 'VISp4VISl4': (0, 48, 12, 38), 'VISp4VISrl4': (0, 61, 0, 45), 'VISp4VISli4': (8, 42, 6, 44), 'VISp4VISpl4': (9, 69, 19, 31), 'VISp4VISal4': (7, 46, 10, 34), 'VISp4VISpor4': (9, 38, 19, 25), 'VISp2/3VISp5': (0, 78, 0, 50), 'VISp2/3VISl4': (0, 48, 12, 38), 'VISp2/3VISrl4': (0, 61, 0, 45), 'VISp2/3VISli4': (9, 40, 7, 43), 'VISp2/3VISpl4': (9, 69, 19, 31), 'VISp2/3VISal4': (8, 44, 11, 32), 'VISp2/3VISpor4': (9, 38, 19, 25), 'VISl4VISl2/3': (0, 39, 0, 26), 'VISl4VISpor4': (10, 29, 0, 22), 'VISrl4VISrl2/3': (0, 48, 0, 36), 'VISrl4VISpor4': (8, 32, 22, 14), 'VISli4VISli2/3': (0, 24, 0, 34), 'VISli4VISpor4': (0, 24, 4, 25), 'VISpl4VISpl2/3': (0, 61, 0, 23), 'VISpl4VISpor4': (0, 23, 0, 10), 'VISal4VISal2/3': (0, 30, 0, 18), 'VISal4VISpor4': (1, 24, 8, 10), 'VISpor4VISpor2/3': (0, 22, 0, 9), 'VISp5VISl4': (0, 48, 12, 38), 'VISp5VISrl4': (0, 61, 0, 45), 'VISp5VISli4': (8, 42, 6, 44), 'VISp5VISpl4': (9, 69, 

In [ ]:
if preprocess_dataset:    
    zipped_dir = Path(r'C:\Users\canda\MouseNet\Assets\RecordedTrials')
    temp_dir = Path("temp_extract")  
    output_dir = Path("PreprocessedTrials")
    output_dir.mkdir(parents=True, exist_ok=True)

    for zip_path in sorted(zipped_dir.glob("batch_*.zip")):
        print(f"📦 Processing {zip_path.name}")
        # Extract to temp
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)

        for trial_dir in temp_dir.glob("trial_*"):
            if not trial_dir.is_dir():
                continue  # Skip .meta or other non-directories
            try:
                # Load and preprocess image
                image_path = trial_dir / "frame_0000.png"
                image = Image.open(image_path).convert("L").resize((64, 64))
                image_tensor = torch.tensor(np.array(image) / 255.0, dtype=torch.float32).unsqueeze(0)  # [1, 60, 60]

                # Load whisker CSVs
                whisker_data = []
                for csv_file in sorted(trial_dir.glob("R*.csv")):
                    df = pd.read_csv(csv_file, header=None).to_numpy()  # [15, 4]
                    whisker_data.append(df)

                whisker_tensor = torch.tensor(np.stack(whisker_data), dtype=torch.float32)  # [60, 15, 4]

                # Save to .pt
                trial_id = trial_dir.name.replace("trial_", "")
                save_path = output_dir / f"trial_{trial_id}.pt"
                torch.save((image_tensor, whisker_tensor), save_path)

            except Exception as e:
                print(f"❌ Failed to process {trial_dir}: {e}")

        # Clean up temp folder
        shutil.rmtree(temp_dir)

    print("✅ Done processing all zipped batches.")

In [5]:
dataset = PreprocessedTrialDataset("PreprocessedTrials")
dataloader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4,   
    pin_memory=True   
)

In [6]:
torch.manual_seed(42)

# --- Hyperparameters ---
lr = 1e-4
epochs = 30
batch_size = 32

# --- Model and Optimizer ---
optimizer = optim.Adam(model.parameters(), lr=lr)

# --- Logging ---
tau_log = []
loss_log = []

model.train()
for epoch in range(epochs):
    total_loss = 0
    for images, whiskers in dataloader:
        images, whiskers = images.to(device), whiskers.to(device)
        optimizer.zero_grad()
        z_v, z_w = model(images, whiskers)
        temp = model.temperature  # already clamped
        loss = clip_loss(z_v, z_w, temperature=temp)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    tau_value = model.temperature.item()

    # Logging
    tau_log.append(tau_value)
    loss_log.append(avg_loss)

    print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}, τ = {tau_value:.4f}")

# Save model and results
model_path = f"models/model_learnable_temp.pt"
torch.save(model.state_dict(), model_path)

model.eval()
all_z_v, all_z_w = [], []

with torch.no_grad():
    for img, whisker in dataloader:
        z_v, z_w = model(img.to(device), whisker.to(device))
        all_z_v.append(z_v.cpu())
        all_z_w.append(z_w.cpu())

torch.save({
    'z_v': torch.cat(all_z_v, dim=0),
    'z_w': torch.cat(all_z_w, dim=0),
    'temperature': model.temperature.item()  # or model.log_temp.exp().item()
}, 'embeddings/embeddings_learnable_temp.pt')



Epoch 1: Loss = 3.2397, τ = 0.1978
Epoch 2: Loss = 2.9374, τ = 0.1964
Epoch 3: Loss = 2.7923, τ = 0.1950
Epoch 4: Loss = 2.6618, τ = 0.1931
Epoch 5: Loss = 2.5531, τ = 0.1907
Epoch 6: Loss = 2.4584, τ = 0.1882
Epoch 7: Loss = 2.3741, τ = 0.1852
Epoch 8: Loss = 2.3049, τ = 0.1822
Epoch 9: Loss = 2.2130, τ = 0.1790
Epoch 10: Loss = 2.1250, τ = 0.1755
Epoch 11: Loss = 2.0473, τ = 0.1722
Epoch 12: Loss = 1.9648, τ = 0.1688
Epoch 13: Loss = 1.8679, τ = 0.1654
Epoch 14: Loss = 1.7830, τ = 0.1620
Epoch 15: Loss = 1.6948, τ = 0.1587
Epoch 16: Loss = 1.6103, τ = 0.1555
Epoch 17: Loss = 1.4976, τ = 0.1523
Epoch 18: Loss = 1.4179, τ = 0.1492
Epoch 19: Loss = 1.3413, τ = 0.1462
Epoch 20: Loss = 1.2355, τ = 0.1433
Epoch 21: Loss = 1.1878, τ = 0.1406
Epoch 22: Loss = 1.0866, τ = 0.1379
Epoch 23: Loss = 1.0260, τ = 0.1354
Epoch 24: Loss = 0.9538, τ = 0.1329
Epoch 25: Loss = 0.8947, τ = 0.1306
Epoch 26: Loss = 0.8433, τ = 0.1284
Epoch 27: Loss = 0.8146, τ = 0.1263
Epoch 28: Loss = 0.7826, τ = 0.1242
E

In [7]:
with open("logs/tau_tracking.json", "w") as f:
    json.dump({
        "tau": tau_log,
        "loss": loss_log
    }, f)

### old code - training with fixed values of temperature

In [ ]:
def train_with_temperature(temp):
    print(f"Training with temperature={temp}")
    torch.manual_seed(42)

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, whiskers in dataloader:
            images, whiskers = images.to(device), whiskers.to(device)
            optimizer.zero_grad()
            z_v, z_w = model(images, whiskers)
            loss = clip_loss(z_v, z_w, temperature=temp)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}")

    # Save model and results
    model_path = f"models/model_temp_{temp:.2f}.pt"
    torch.save(model.state_dict(), model_path)

    model.eval()
    all_z_v, all_z_w = [], []

    with torch.no_grad():
        for img, whisker in dataloader:
            z_v, z_w = model(img.to(device), whisker.to(device))
            all_z_v.append(z_v.cpu())
            all_z_w.append(z_w.cpu())

    torch.save({
        'z_v': torch.cat(all_z_v, dim=0),
        'z_w': torch.cat(all_z_w, dim=0)
    }, f'embeddings/embeddings_temp_{temp:.2f}.pt')

for temp in [0.15, 0.2]:
    model = MultimodalMouseModelPool(visual_net=visual_net).to(device)  # Re-initialize for each temperature
    optimizer = optim.Adam(model.parameters(), lr=lr)
    train_with_temperature(temp)
